# Final Project -- Part 2
## Oliver Quarm (oquarm2)

## Section 1: Code
### Running instructions: 
I've tried opening this Jupyter Notebook in VSCode and it doesn't appear correctly. Run the Notebook as a regular Jupyter Notebook server (through your browser) and the widgets will display. I used Python 3.7 as my kernel for this Notebook. 
### Import neccessary packages

In [10]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

### Read in the data

In [11]:
df = pd.read_csv("./athlete_events.csv")
# df = df[df['Medal'] == "Gold"]
df = df[df['Team'].notna()]
df = df[df['Medal'].notna()]
df = df[df['Height'].notna()]
df = df[df['Weight'].notna()]
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
40,16,Juhamatti Tapio Aaltonen,M,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
41,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
42,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold
44,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold
48,17,Paavo Johannes Aaltonen,M,28.0,175.0,64.0,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Pommelled Horse,Gold


### Select out the columns we need for the widgets.

In [12]:
(
    df
    .groupby(['Sex', 'Team', 'Age', 'Height', 'Weight', 'Year'])['Medal'].count()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
).head(5)

,Sex,Team,Age,Height,Weight,Year,Medal
23990,M,United States,20.0,188.0,81.0,1896,4
25325,M,United States,27.0,175.0,72.0,1896,3
24120,M,United States,21.0,183.0,66.0,1896,2
23914,M,United States,20.0,173.0,69.0,1896,1
14534,M,Germany,24.0,167.0,56.0,1896,2


### Create widget set #1

In [13]:
idf = df.interactive()

In [14]:
year = pn.widgets.IntSlider(name='Year', start=1896, end=2016, step=4, bar_color='#bbc2f0')
sex_widget = pn.widgets.ToggleGroup(
    name='Sex',
    options=['M', 'F'], 
    value=['M', 'F'],
    button_type='primary')
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Medal'],
    button_type='primary'
)

In [15]:
ipipeline = (
    idf[
        (idf.Year == year) & 
        (idf.Sex.isin(sex_widget))
    ]
    .groupby(['Team'])[yaxis].count()
    .to_frame()
    .reset_index()
    .sort_values(by='Team')  
    .reset_index(drop=True)
)
ipipeline.head(10) # Change this number to determine how many teams you want to display at a time.

### Explanation
The above is the `ipipeline` first visualized. By default, both genders are preselected. Click a gender button to remove them (or later add them) to/from the count. Below the data/widget is piped to a table for better viewing. You don't need to click the buttons in the below table -- click the buttons above and the output below will also change accordingly! The beauty of piping to an `itable` as per below is that you can click the sort columns arrows to sort each column (that's the main benefit of piping the data/widgets to a table below).

In [16]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)
itable

### Create widget set #2
This widget set allows you to explore the average weight and mean for each Olympic team. And this is per year -- adjust the slider to see how this changes over time. You'll notice for the earlier years (e.g. 1896) there are only a limited number of teams displayed as the height & weight info wasn't known for many atheletes during these early times. Click between the `Height` and `Weight` buttons to view each independently.

In [17]:
idf2 = df.interactive()

year = pn.widgets.IntSlider(name='Year', start=1896, end=2016, step=4, bar_color='#f0d17d')
sex_widget = pn.widgets.ToggleGroup(
    name='Sex',
    options=['M', 'F'], 
    value=['M', 'F'],
    button_type='success')
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Height', 'Weight'],
    button_type='success'
)

ipipeline2 = (
    idf2[
        (idf2.Year == year) & 
        (idf2.Sex.isin(sex_widget))
    ]
    .groupby(['Team'])[yaxis].mean()
    .to_frame()
    .sort_values(by='Team')
)
ipipeline2.head(10) # Change this number to determine how many teams you want to display at a time.

In [18]:
itable = ipipeline2.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)
itable

## Section 2: Write-up

### GitHub
My dataset is not a 'larger' one as it is within the limits specified by GitHub.

### Contextual Datasets
I found a dataset specifically on the Tokyo 2021 Olympics. This dataset is more indepth than the one explored in this Notebook, as this dataset is more generalized, as the data is of athletes competing in every single Olympic Games since 1896. This new contextual dataset has more metrics, such as details of the athletes, coach information, their discpline and much more. It can be found at: https://www.kaggle.com/datasets/arjunprasadsarkhel/2021-olympics-in-tokyo. It'd be useful as it gives a different perspective of Olympic Games data.

### Dashboard Explanation
**Dashboard #1** allows the user to see each olympic team's medal count over each games (each year). If you move the slider at the top of the dashboard across, you can see how the medal counts change over time -- so you can see for a specific Olympic Games, how many medals each team got. 

There's also functionality involving gender. The medal count per gender can be seen by clicking either the 'M' button, 'F' button, or both at the same time! Depending on which one is pressed, it will display the medal count each gender contributes. For example, if you move the slider to the year 2000, we can see Australia got a total of 181 medals. Selecting the M button gives 94, whilst the F button gives 87 -- both genders seem to contribute almost evenly for Australia's medal count!

Piping the data to the `itable` allows us to sort each column, in ascending or descending order. This is particulary useful for seeing, for a given year, the top 10 countries that got the most medals, or even the countries that got the least amount of medals!

**Dashboard #2** allows you to see the average weight and height for athletes for each team over time. The functionality is similar to Dashboard #1, so the functionality exists over all the games (1896 onwards), and you can filter it per gender. In this dashboard the `index` column was dropped just to show that it is possible to omit this column if desired.

***Note***: I was originally part of Group 10. One of my teammates dropped out of the group last minute; as a result I've completed Part 2 by myself.